In [1]:
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import random
import numpy as np
import pylab
import quandl
quandl.ApiConfig.api_key = "32peB6ExRM7Lihbpsdsd"
import seaborn as sns
sns.set()
%matplotlib inline
#textblob to apply sentiment analysis 
from textblob import TextBlob
import preprocessor as p

In [7]:
#read the csv file containing tweets about 'Tesla'
tweets = pd.read_csv('../using_twint/tesla_tweets_3.csv', low_memory=False)
tweets = tweets[['date','tweet']]#Collapse dataframe to only the variables we want
tweets.tail(10)

,date,tweet
45551,2014-12-07,"The Tesla Bet is on! @Lord_Kat & I, first one ..."
45552,2014-12-07,Uber's $40 billion value is bigger than (marke...
45553,2014-12-07,"In two decades, there will be 1 billion more c..."
45554,2014-12-07,Tesla's CTO Wants To Take Batteries To A New D...
45555,2014-12-07,#Tesla #Investors Are Ignoring What #ElonMusk ...
45556,2014-12-07,JB Straubel is charged with taking batteries t...
45557,2014-12-07,Tesla's CTO Wants To Take Batteries To A New D...
45558,2014-12-07,Watch how to make music from a Tiny tesla coil...
45559,2014-12-07,Great article by @markchediak on the downstrea...
45560,2014-12-07,Tesla's CTO Wants To Take Batteries To A New D...


In [37]:
#get a particular tweet text
some_tweet = tweets.iloc[4551]['tweet']
print(some_tweet,"\n", p.clean(some_tweet))

#get the 'sentiment' of a particular tweet 
analysis = TextBlob(some_tweet)
print(analysis.sentiment)

Tesla shares spike after vote of confidence from Elon Musk $tsla  http://nyp.st/1Onk2Qd  via @nypost via @jamescovert1 
 Tesla shares spike after vote of confidence from Elon Musk $tsla via via
Sentiment(polarity=0.0, subjectivity=0.0)


In [35]:
tweets['polarity'] = tweets['tweet'].apply(lambda x:TextBlob(x).polarity)#p.clean() removes links, emojis, hastags
tweets['subjectivity'] = tweets['tweet'].apply(lambda x:TextBlob(x).subjectivity)
tweets['tweet_clean'] = tweets['tweet'].apply(lambda x:p.clean(x))

In [36]:
tweets.tail(10)

,date,tweet,polarity,subjectivity,tweet_clean
45551,2014-12-07,"The Tesla Bet is on! @Lord_Kat & I, first one ...",0.062500,0.354167,"The Tesla Bet is on! & I, first one to get the..."
45552,2014-12-07,Uber's $40 billion value is bigger than (marke...,0.000000,0.250000,Uber's $40 billion value is bigger than (marke...
45553,2014-12-07,"In two decades, there will be 1 billion more c...",0.500000,0.500000,"In two decades, there will be billion more car..."
45554,2014-12-07,Tesla's CTO Wants To Take Batteries To A New D...,0.168182,0.277273,Tesla's CTO Wants To Take Batteries To A New D...
45555,2014-12-07,#Tesla #Investors Are Ignoring What #ElonMusk ...,0.200000,0.200000,Are Ignoring What Really Said About BMW:
45556,2014-12-07,JB Straubel is charged with taking batteries t...,0.136364,0.454545,JB Straubel is charged with taking batteries t...
45557,2014-12-07,Tesla's CTO Wants To Take Batteries To A New D...,0.168182,0.277273,Tesla's CTO Wants To Take Batteries To A New D...
45558,2014-12-07,Watch how to make music from a Tiny tesla coil...,0.250000,0.450000,Watch how to make music from a Tiny tesla coil...
45559,2014-12-07,Great article by @markchediak on the downstrea...,0.800000,0.750000,Great article by on the downstream Gigafactory...
45560,2014-12-07,Tesla's CTO Wants To Take Batteries To A New D...,0.168182,0.277273,Tesla's CTO Wants To Take Batteries To A New D...


In [ ]:
#covert the date column to datetime type
tweets.date = pd.to_datetime(tweets.date)
#append weekofyear, weekday, year based on date, 0=Mon, 6=Sun
tweets['weekofyear'] = tweets['date'].dt.weekofyear
tweets['weekday'] = tweets['date'].dt.weekday
tweets['year'] = tweets['date'].dt.year

In [ ]:
columns = ['polarity', 'weekofyear']
weekly_pol = tweets.loc[tweets['year'] == 2015, columns].groupby('weekofyear').mean()
weekly_pol = weekly_pol[:-1]

In [ ]:
#plot the weekly average sentiment for 2015
pylab.rcParams['figure.figsize'] = (15, 9)
plt.ylim(-0.25, 0.25)
plt.plot(np.arange(0,52), weekly_pol['polarity'])

In [ ]:
#get the weekly percent difference in the stock prices 
start = datetime.datetime(2015,1,1) # Beginning of period 
end = datetime.datetime(2016,1,1) # End of period 
stock = "TSLA" # some stock label we are interested in
data = quandl.get("WIKI/" + stock, start_date=start, end_date=end)
pylab.rcParams['figure.figsize'] = (13, 7)
plt.title(stock + " Closing Prices")
plt.plot(data["Close"],label="Daily closing price") # plot daily closing prices 
plt.legend()
plt.show()

In [ ]:
#get the weekly percent difference in the stock price
data["WeekPercDiff"] = data["Close"].diff(periods=5) / data["Close"]
plt.title(stock + " Weekly Perent Difference")
plt.plot(data["WeekPercDiff"],label="Weekly Percent Diff") # plot daily closing prices 
plt.legend()
plt.show()

In [ ]:
data['weekofyear'] = data.index.weekofyear
data['weekday'] = data.index.weekday
data['year'] = data.index.year

In [ ]:
#get price percent differences on fridays (weekday == 4)
wed_stocks = data.loc[data['weekday'] == 2]
wed_stocks.head()

In [ ]:
#weekly_pol['polarity'].shape
#wed_stocks["WeekPercDiff"].shape
plt.title("Polarity and Weekly Perent Price Differece")
plt.xlabel('Weekly Average Polarity')
plt.ylabel('Weekly Price Difference')
plt.scatter(weekly_pol['polarity'], wed_stocks["WeekPercDiff"])
plt.show()

In [ ]:
plt.title("Distribution of Polarity")
plt.xlabel("Polarity")
#plt.hist(weekly_pol['polarity'])
sns.distplot(weekly_pol['polarity'])
plt.show()

In [ ]:
plt.title("Distribution of Relative Price Differences")
plt.xlabel("Weekly Price Percent Diff")
#sns.distplot(wed_stocks["WeekPercDiff"])
plt.hist(wed_stocks["WeekPercDiff"])

In [ ]:
from sklearn.linear_model import LinearRegression
weekly_pol['polarity']=weekly_pol['polarity'].values.reshape(-1,1) 
wed_stocks["WeekPercDiff"] = wed_stocks["WeekPercDiff"].values.reshape(-1,1) 
wed_stocks["WeekPercDiff"] = wed_stocks["WeekPercDiff"].fillna(0)
lm = LinearRegression().fit( weekly_pol[['polarity']], wed_stocks["WeekPercDiff"])


In [ ]:
plt.title("Polarity and Weekly Perent Price Differece")
plt.xlabel('Weekly Average Polarity')
plt.ylabel('Weekly Price Difference')
plt.scatter(weekly_pol['polarity'], wed_stocks["WeekPercDiff"])
plt.plot( weekly_pol['polarity'], lm.predict(weekly_pol[['polarity']]), color='red', linewidth=2 )


In [ ]:
 from sklearn.model_selection import KFold
model = LinearRegression()
scores = []

X = weekly_pol[['polarity']]
y = wed_stocks["WeekPercDiff"]

kf = KFold(n_splits=2, shuffle=True, random_state=1)
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    model.fit(X_train, y_train)
    score = model.score(X_test, y_test)
    scores.append(score)

scores